# Smart Aliens:
## Using Statistical Interference to Find Exoplanets that Could Harbor Intelligent Alien Life
---

### by Julian Estrada & William Walker

- 
 

In [1]:
#### IMPORT Everything here ####
import ast
import bs4
import flask
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import requests
import scipy
import scipy.stats as st
import sqlalchemy




from bs4 import BeautifulSoup

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

#from selenium import webdriver
#from splinter import Browser

# Part I: Using Flask

In [2]:
## API Flask ##

## Import via loading in csv file until API works

 

In [10]:
## Read in csv file ##
exoplanet_metadata_path = "data/exoplanet-archive.csv"
exoplanet_metadata = pd.read_csv(exoplanet_metadata_path)


/var/folders/hf/mp5v5rls3hg252gfcnygxxpw0000gn/T/ipykernel_98460/3010382786.py:3: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  exoplanet_metadata = pd.read_csv(exoplanet_metadata_path)


# Part II: Cleaning the Data

In [11]:
## Create exoplanet dataframe with columns we want ##
exoplanet_df = exoplanet_metadata[[
    "pl_name", "hostname", "disc_year", "discoverymethod", "pl_orbper", "pl_orbsmax",\
          "pl_rade", "pl_masse", "pl_bmasse", "pl_insol", "pl_eqt", "st_spectype", "st_teff",\
              "glat", "glon", "sy_dist"
]]


In [12]:
## Shortcut of Math Items ##
pi = np.pi

sin = np.sin
cos = np.cos
arcsin = np.arcsin
arccos = np.arccos
sqrt = np.sqrt

In [13]:

## Convert distance to Light-Years ##
lightyear = ((2 ** 2) * 3 * (5 ** 3) * 877 * 7789 ) / (pi * 7 * 487 * 293339)
exoplanet_df["sy_dist"] *= lightyear

## Experimental average gravity of Earth in m/s^2 ##
g_earth = 9.8203

## Insert Gravity columns ##
exoplanet_df["pl_gravitye"] = exoplanet_df["pl_masse"] / (exoplanet_df["pl_rade"] ** 2)
exoplanet_df["pl_gravity"] = round(g_earth * exoplanet_df["pl_gravitye"], 4)

## X and Y value coordinates in Light-Years ##
exoplanet_df["xVal"] = exoplanet_df["sy_dist"] * -sin( exoplanet_df["glon"] * pi / 180 )\
      * cos( exoplanet_df["glat"] * pi / 180 )
exoplanet_df["yVal"] = exoplanet_df["sy_dist"] * cos( exoplanet_df["glon"] * pi / 180 )\
      * cos( exoplanet_df["glat"] * pi / 180 )



/var/folders/hf/mp5v5rls3hg252gfcnygxxpw0000gn/T/ipykernel_98460/767550919.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exoplanet_df["sy_dist"] *= lightyear
/var/folders/hf/mp5v5rls3hg252gfcnygxxpw0000gn/T/ipykernel_98460/767550919.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exoplanet_df["pl_gravitye"] = exoplanet_df["pl_masse"] / (exoplanet_df["pl_rade"] ** 2)
/var/folders/hf/mp5v5rls3hg252gfcnygxxpw0000gn/T/ipykernel_98460/767550919.py:10: SettingWithCopyWarning: 
A value is trying to be se

In [14]:
## Reorder Columns ##
exoplanet_df = exoplanet_df[[
    "pl_name", "hostname", "disc_year", "discoverymethod", "pl_orbper", "pl_orbsmax",\
          "pl_rade", "pl_masse", "pl_gravitye", "pl_gravity", "pl_insol", "pl_eqt",\
              "st_spectype", "st_teff", "glat", "glon", "sy_dist", "xVal", "yVal"
]]

## Drop all rows with Null values ##
exoplanet_df = exoplanet_df.dropna()
plNum = len(exoplanet_df["pl_name"].unique())

## Drop all rows with repeated planets ##
exoplanet_df = exoplanet_df.drop_duplicates(subset=["pl_name", "pl_rade"], keep= False)

## How many Planets do we have? ##
print(f"How many unique Exoplanets have Full Data? It's {plNum}!")



exoplanet_df.head(25)


How many unique Exoplanets have Full Data? It's 172!


,pl_name,hostname,disc_year,discoverymethod,pl_orbper,pl_orbsmax,pl_rade,pl_masse,pl_gravitye,pl_gravity,pl_insol,pl_eqt,st_spectype,st_teff,glat,glon,sy_dist,xVal,yVal
343,CoRoTID 223977153 b,CoRoTID 223977153,2017,Transit,6.718370,0.07100,6.389,47.67450,1.167940,11.4695,132.26,938.0,G0 VI,5970.0,1.86373,203.48237,1880.888384,749.075238,-1724.205756
472,EPIC 246851721 b,EPIC 246851721,2018,Transit,6.180235,0.07229,11.781,953.49000,6.869922,67.4647,642.00,1401.0,F5 V,6202.0,-12.64088,186.97531,1215.363033,144.018068,-1177.126315
474,EPIC 249893012 b,EPIC 249893012,2020,Transit,3.595100,0.04700,1.950,8.75000,2.301118,22.5977,1037.00,1616.0,G8 IV/V,5430.0,34.30816,345.11043,1047.927395,222.423647,836.542327
475,EPIC 249893012 c,EPIC 249893012,2020,Transit,15.624000,0.13000,3.670,14.67000,1.089176,10.6960,160.00,990.0,G8 IV/V,5430.0,34.30816,345.11043,1047.927395,222.423647,836.542327
476,EPIC 249893012 d,EPIC 249893012,2020,Transit,35.747000,0.22000,3.940,10.18000,0.655776,6.4399,53.00,752.0,G8 IV/V,5430.0,34.30816,345.11043,1047.927395,222.423647,836.542327
481,G 9-40 b,G 9-40,2019,Transit,5.746007,0.03850,2.025,11.70000,2.853224,28.0195,7.20,456.0,M2.5 V,3348.0,37.03563,206.06509,91.087649,31.948946,-65.316615
506,GJ 1214 b,GJ 1214,2009,Transit,1.580404,0.01490,2.742,8.17000,1.086644,10.6712,21.00,596.0,M4 V,3250.0,23.60704,26.16217,47.758100,-19.294982,39.278025
600,GJ 357 b,GJ 357,2019,Transit,3.930720,0.03500,1.217,1.84000,1.242329,12.2000,12.60,525.0,M2.5 V,3505.0,22.08045,253.88843,30.795065,27.415639,-7.919115
604,GJ 367 b,GJ 367,2021,Transit,0.321923,0.00709,0.699,0.63300,1.295536,12.7225,579.00,1365.0,M1.0 V,3522.0,5.65890,272.24323,30.699893,30.526867,1.195791
612,GJ 3929 b,GJ 3929,2022,Transit,2.616274,0.02569,1.150,1.21000,0.914934,8.9849,17.50,569.0,M3.5 V,3369.0,49.59735,56.66419,51.563693,-27.922271,18.366496


In [131]:
## Name of File: ##
file_name = 'exoplanet_df'
## Upload Dataframe to csv file ##
exoplanet_df.to_csv(f"data/{file_name}.csv", sep=',', encoding='utf-8')

In [21]:

print(exoplanet_df["xVal"].min())
print(exoplanet_df["xVal"].max())
print("spacespace")
print(exoplanet_df["yVal"].min())
print(exoplanet_df["yVal"].max())

-2027.9009248325199
2977.649629767964
spacespace
-1724.205755536837
2567.6096682958228


In [35]:
xValList = list(exoplanet_df["xVal"])
yValList = list(exoplanet_df["yVal"])

planetNameList = list(exoplanet_df["pl_name"])

for i in range(0, len(xValList)):
    xValList[i] = round(xValList[i], 3)

for j in range(0, len(yValList)):
    yValList[j] = round(yValList[j], 3)


print(xValList)
print(yValList)
print(planetNameList)

[749.075, 144.018, 222.424, 222.424, 222.424, 31.949, -19.295, 27.416, 30.527, -27.922, 6.817, -51.833, -51.833, -51.833, -51.833, -51.833, -51.833, -1456.25, 177.781, 1062.842, 1698.211, -705.154, 282.406, -164.028, -164.028, -164.028, 45.089, -74.606, 89.584, 89.584, 89.584, 89.584, 89.584, 8.783, 8.783, -251.819, 131.566, 83.561, 83.561, 59.347, 59.347, 114.638, 65.867, 222.388, 618.287, 618.287, 337.922, -179.195, 91.23, 91.23, -4.035, -295.366, -295.366, -295.366, -295.366, 142.018, 81.621, 81.621, 81.621, 104.815, 104.815, 104.815, 104.815, 35.37, 103.275, -603.736, 480.648, -404.56, -1455.401, -289.727, 173.811, -654.391, -365.414, -1682.303, -1225.19, 32.316, 32.316, 32.316, -9.141, 67.03, 67.03, 54.672, 54.672, 55.594, 55.594, 55.594, 55.594, 533.205, 1251.673, 1651.779, 1418.568, 2210.713, 2977.65, -631.762, 1735.008, -2027.901, -907.961, -988.008, 189.497, 81.369, 757.785, 15.193, 87.923, -60.834, -437.226, -437.226, -437.226, -437.226, 209.06, 209.06, 209.06, -101.946, -101

In [39]:
insolValList = list(exoplanet_df["pl_insol"])
tempValList = list(exoplanet_df["pl_eqt"])

for k in range(0, len(insolValList)):
    insolValList[k] *= 1361

print(insolValList)
print(tempValList)

[180005.86, 873762.0, 1411357.0, 217760.0, 72133.0, 9799.2, 28581.0, 17148.6, 788019.0, 23817.5, 54848.299999999996, 348416.0, 428715.0, 80299.0, 97992.0, 48451.6, 39469.0, 6328650.0, 251512.80000000002, 98944.7, 4069390.0, 763521.0, 1001001.89, 136100.0, 28581.0, 19462.3, 99216.90000000001, 36066.5, 21776.0, 10833.56, 159237.0, 63558.700000000004, 35794.3, 57434.200000000004, 21912.100000000002, 133500.49, 179652.0, 58931.299999999996, 13882.199999999999, 3944178.0, 740111.7999999999, 10098.62, 899621.0, 2606315.0, 277644.0, 256548.5, 91187.0, 337528.0, 81660.0, 32664.0, 13487.51, 318895.91, 68526.35, 33494.21, 14290.5, 91187.0, 14290.5, 4314.37, 1959.84, 634089.9, 579786.0, 174616.30000000002, 159237.0, 6560.02, 816600.0, 4289872.0, 3184740.0, 5937362.5, 4014950.0, 1348995.98, 2504240.0, 61108900.0, 992169.0, 408.3, 1197.68, 33616.7, 17420.8, 6818.61, 626.0600000000001, 3593.04, 7934.63, 5566.49, 1238.51, 157876.0, 144266.0, 3579.43, 3919.68, 88873.3, 566176.0, 578425.0, 653280.0, 15

In [40]:
radiiValList = list(exoplanet_df["pl_rade"])
massValList = list(exoplanet_df["pl_masse"])



print(radiiValList)
print(massValList)



[6.389, 11.781, 1.95, 3.67, 3.94, 2.025, 2.742, 1.217, 0.699, 1.15, 1.305, 1.58, 1.62, 1.24, 1.269, 2.07, 2.04, 16.477, 12.016, 8.709, 14.986, 13.843, 2.05, 3.41, 3.195, 2.995, 2.25, 2.501, 2.0, 1.87, 0.75, 0.818, 1.137, 1.24, 1.533, 9.718, 2.45, 2.79, 2.39, 1.35, 3.228, 3.89, 22.754, 3.88, 13.563, 12.274, 2.242, 1.75, 5.68, 7.82, 3.44, 2.59, 3.53, 2.48, 1.95, 2.63, 2.078, 1.582, 1.458, 1.55, 1.54, 2.42, 2.29, 1.015, 15.681, 17.598, 21.409, 19.515, 17.777, 14.83, 19.044, 21.196, 11.545, 9.18, 3.87, 0.85, 1.385, 1.521, 1.43, 2.438, 1.032, 1.32, 1.367, 1.35, 1.332, 2.3, 2.42, 14.908, 11.747, 12.33, 14.572, 13.899, 13.563, 12.218, 11.994, 12.285, 12.722, 12.408, 2.265, 1.791, 14.572, 2.415, 3.65, 1.694, 3.01, 2.51, 3.51, 3.78, 2.726, 2.759, 2.93, 2.34, 2.82, 2.458, 2.37, 1.673, 1.56, 3.42, 1.62, 16.701, 1.672, 2.18, 2.47, 14.415, 1.79, 12.173, 1.9, 5.05, 3.14, 16.141, 6.956, 12.89, 1.25, 1.33, 2.19, 11.321, 3.51, 1.52, 18.103, 2.77, 8.295, 11.321, 12.0, 5.14, 6.0, 7.9, 3.02, 5.93, 6.18, 1

# Part III: Further manipulation

In [123]:
#### Get most Earth-Like planets ####

## Sort by earth radii ##
earthRade = 1
topRadii = exoplanet_df.iloc[(exoplanet_df['pl_rade'] - earthRade).abs().argsort()[:]]

## Sort by earth mass ##
earthMass = 1
topMass = exoplanet_df.iloc[(exoplanet_df['pl_masse'] - earthMass).abs().argsort()[:]]

## Sort by earth black body temp ##
earthEQT = 255
topEQT = exoplanet_df.iloc[(exoplanet_df['pl_eqt'] - earthEQT).abs().argsort()[:]]



In [124]:
topRadii.head(10)


,pl_name,hostname,disc_year,discoverymethod,pl_orbper,pl_orbsmax,pl_rade,pl_masse,pl_gravitye,pl_gravity,pl_insol,pl_eqt,st_spectype,st_teff,glat,glon,sy_dist,xVal,yVal
5014,K2-415 b,K2-415,2023,Transit,4.017969,0.02700,1.015,7.50,7.279963,71.4914,4.82,412.0,M5 V,3173.0,35.80871,217.79828,71.161451,35.369589,-45.601026
32310,LP 791-18 d,LP 791-18,2023,Transit,2.753436,0.01992,1.032,0.90,0.845051,8.2987,5.83,396.0,M(6.1 +/- 0.7) V,2960.0,39.10605,268.56582,86.407631,67.029572,-1.678179
2725,HD 23472 f,HD 23472,2022,Transit,12.162184,0.09060,1.137,0.77,0.595621,5.8492,26.30,630.0,K4 V,4684.0,-44.84798,277.02690,127.312207,89.583889,11.042214
612,GJ 3929 b,GJ 3929,2022,Transit,2.616274,0.02569,1.150,1.21,0.914934,8.9849,17.50,569.0,M3.5 V,3369.0,49.59735,56.66419,51.563693,-27.922271,18.366496
32264,L 98-59 b,L 98-59,2019,Transit,2.253114,0.02191,0.850,0.40,0.553633,5.4368,24.70,627.0,M3 V,3415.0,-17.59330,281.81705,34.635850,32.316073,6.761216
33487,TOI-500 b,TOI-500,2022,Transit,0.548177,0.01189,1.166,1.42,1.044459,10.2569,1138.00,1617.0,K6 V,4440.0,-17.33308,258.09657,154.573335,144.381091,-30.434895
2723,HD 23472 e,HD 23472,2022,Transit,7.907540,0.06800,0.818,0.72,1.076034,10.5670,46.70,723.0,K4 V,4684.0,-44.84798,277.02690,127.312207,89.583889,11.042214
600,GJ 357 b,GJ 357,2019,Transit,3.930720,0.03500,1.217,1.84,1.242329,12.2000,12.60,525.0,M2.5 V,3505.0,22.08045,253.88843,30.795065,27.415639,-7.919115
781,GJ 9827 c,GJ 9827,2017,Transit,3.648230,0.04290,1.240,1.45,0.943028,9.2608,59.00,771.0,K6 V,4219.0,-57.18299,81.34803,96.741243,-51.833033,7.887106
2756,HD 260655 b,HD 260655,2022,Transit,2.769530,0.02933,1.240,2.14,1.391779,13.6677,42.20,709.0,M0 V,3803.0,4.86549,195.67069,32.633576,8.782825,-31.307366


In [125]:
topMass.head(10)


,pl_name,hostname,disc_year,discoverymethod,pl_orbper,pl_orbsmax,pl_rade,pl_masse,pl_gravitye,pl_gravity,pl_insol,pl_eqt,st_spectype,st_teff,glat,glon,sy_dist,xVal,yVal
32310,LP 791-18 d,LP 791-18,2023,Transit,2.753436,0.01992,1.032,0.900,0.845051,8.2987,5.83,396.0,M(6.1 +/- 0.7) V,2960.0,39.10605,268.56582,86.407631,67.029572,-1.678179
612,GJ 3929 b,GJ 3929,2022,Transit,2.616274,0.02569,1.150,1.210,0.914934,8.9849,17.50,569.0,M3.5 V,3369.0,49.59735,56.66419,51.563693,-27.922271,18.366496
2725,HD 23472 f,HD 23472,2022,Transit,12.162184,0.09060,1.137,0.770,0.595621,5.8492,26.30,630.0,K4 V,4684.0,-44.84798,277.02690,127.312207,89.583889,11.042214
2723,HD 23472 e,HD 23472,2022,Transit,7.907540,0.06800,0.818,0.720,1.076034,10.5670,46.70,723.0,K4 V,4684.0,-44.84798,277.02690,127.312207,89.583889,11.042214
604,GJ 367 b,GJ 367,2021,Transit,0.321923,0.00709,0.699,0.633,1.295536,12.7225,579.00,1365.0,M1.0 V,3522.0,5.65890,272.24323,30.699893,30.526867,1.195791
33487,TOI-500 b,TOI-500,2022,Transit,0.548177,0.01189,1.166,1.420,1.044459,10.2569,1138.00,1617.0,K6 V,4440.0,-17.33308,258.09657,154.573335,144.381091,-30.434895
2721,HD 23472 d,HD 23472,2022,Transit,3.976640,0.04298,0.750,0.550,0.977778,9.6021,117.00,909.0,K4 V,4684.0,-44.84798,277.02690,127.312207,89.583889,11.042214
781,GJ 9827 c,GJ 9827,2017,Transit,3.648230,0.04290,1.240,1.450,0.943028,9.2608,59.00,771.0,K6 V,4219.0,-57.18299,81.34803,96.741243,-51.833033,7.887106
32264,L 98-59 b,L 98-59,2019,Transit,2.253114,0.02191,0.850,0.400,0.553633,5.4368,24.70,627.0,M3 V,3415.0,-17.59330,281.81705,34.635850,32.316073,6.761216
600,GJ 357 b,GJ 357,2019,Transit,3.930720,0.03500,1.217,1.840,1.242329,12.2000,12.60,525.0,M2.5 V,3505.0,22.08045,253.88843,30.795065,27.415639,-7.919115


In [126]:
topEQT.head(10)


,pl_name,hostname,disc_year,discoverymethod,pl_orbper,pl_orbsmax,pl_rade,pl_masse,pl_gravitye,pl_gravity,pl_insol,pl_eqt,st_spectype,st_teff,glat,glon,sy_dist,xVal,yVal
14352,Kepler-1661 b,Kepler-1661,2020,Transit,175.060000,0.63300,3.870,17.000,1.135081,11.1468,0.88,243.0,K,5100.0,16.15714,72.26462,1339.220918,-1225.190100,391.841809
32313,LP 890-9 c,LP 890-9,2022,Transit,8.457463,0.03984,1.367,25.300,13.538903,132.9561,0.91,272.0,M6 V,2850.0,-44.94830,226.91411,105.771861,54.672146,-51.136041
32281,LHS 1140 b,LHS 1140,2017,Transit,24.737120,0.08750,1.430,6.650,3.251993,31.9355,0.46,230.0,M4.5 V,3131.0,-78.05159,115.40542,48.878121,-9.140692,-4.341379
33238,TOI-2095 c,TOI-2095,2023,Transit,28.172320,0.13700,1.330,7.400,4.183391,41.0822,1.84,297.0,M2.5 V,3759.0,25.49556,106.75582,136.716926,-118.163801,-35.576358
14290,Kepler-1654 b,Kepler-1654,2018,Transit,1047.835600,2.02600,9.180,158.915,1.885730,18.5184,0.30,206.0,G5 V,5597.0,19.06885,73.86663,1852.949828,-1682.302784,486.633355
4613,K2-3 d,K2-3,2015,Transit,44.556030,0.20140,1.458,2.200,1.034922,10.1632,1.44,305.0,M V,3844.0,55.25008,264.98752,143.745922,81.621202,-7.158845
32307,LP 791-18 c,LP 791-18,2019,Transit,4.989909,0.02961,2.438,7.100,1.194513,11.7305,2.64,324.0,M(6.1 +/- 0.7) V,2960.0,39.10605,268.56582,86.407631,67.029572,-1.678179
33099,TOI-1452 b,TOI-1452,2022,Transit,11.062010,0.06100,1.672,4.820,1.724148,16.9316,1.80,326.0,M4 &#177 0.5 V,3185.0,23.81292,104.66497,99.546840,-88.105396,-23.056453
33018,TOI-1231 b,TOI-1231,2021,Transit,24.245586,0.12880,3.650,15.400,1.155939,11.3517,1.93,330.0,M3 V,3553.0,4.38237,281.82560,90.093198,87.923239,18.409124
33056,TOI-1266 c,TOI-1266,2020,Transit,18.801510,0.10580,1.560,2.200,0.904011,8.8777,2.30,344.0,M3,3600.0,51.16066,119.58108,117.454782,-64.059397,-36.362827


In [128]:

planetNameList = []
earthRankList = []


for i in range(0, len(exoplanet_df["pl_name"])):
    planetNameList.append(list(exoplanet_df["pl_name"])[i])

    ## radii arg ##
    radARG = float(list(exoplanet_df["pl_rade"])[i]) - earthRade

    ## mass arg ##
    massARG = float(list(exoplanet_df["pl_masse"])[i]) - earthMass

    ## temp arg ##
    tempARG =  ( float(list(exoplanet_df["pl_eqt"])[i]) - earthEQT) / earthEQT


    ## Average specs out ##
    earthRank = round(100 * np.e ** (-sqrt( radARG ** 2 + massARG ** 2 + tempARG ** 2 )), 3)
    earthRankList.append(earthRank)

earthRankList

earthLike_df = pd.DataFrame({
    "pl_name": planetNameList,
    "earth-like_score": earthRankList
})


earthLike_df = earthLike_df.sort_values("earth-like_score", ascending = False)

earthLike_df.head(20)

,pl_name,earth-like_score
80,LP 791-18 d,56.960
77,L 98-59 d,28.752
9,GJ 3929 b,28.419
58,K2-3 d,27.272
7,GJ 357 b,25.440
116,TOI-1266 c,25.424
32,HD 23472 f,22.430
75,L 98-59 b,20.505
76,L 98-59 c,17.680
57,K2-3 c,15.942


In [129]:
exoplanet_df.loc[exoplanet_df["pl_name"] == 'LP 791-18 d', :]


,pl_name,hostname,disc_year,discoverymethod,pl_orbper,pl_orbsmax,pl_rade,pl_masse,pl_gravitye,pl_gravity,pl_insol,pl_eqt,st_spectype,st_teff,glat,glon,sy_dist,xVal,yVal
32310,LP 791-18 d,LP 791-18,2023,Transit,2.753436,0.01992,1.032,0.9,0.845051,8.2987,5.83,396.0,M(6.1 +/- 0.7) V,2960.0,39.10605,268.56582,86.407631,67.029572,-1.678179


In [15]:
exoplanet_df.loc[exoplanet_df["pl_name"] == 'GJ 3929 b', :]

,pl_name,hostname,disc_year,discoverymethod,pl_orbper,pl_orbsmax,pl_rade,pl_masse,pl_gravitye,pl_gravity,pl_insol,pl_eqt,st_spectype,st_teff,glat,glon,sy_dist,xVal,yVal
612,GJ 3929 b,GJ 3929,2022,Transit,2.616274,0.02569,1.15,1.21,0.914934,8.9849,17.5,569.0,M3.5 V,3369.0,49.59735,56.66419,51.563693,-27.922271,18.366496


In [41]:
exoplanet_df.loc[exoplanet_df["pl_name"] == 'KELT-9 b', :]

,pl_name,hostname,disc_year,discoverymethod,pl_orbper,pl_orbsmax,pl_rade,pl_masse,pl_gravitye,pl_gravity,pl_insol,pl_eqt,st_spectype,st_teff,glat,glon,sy_dist,xVal,yVal
5406,KELT-9 b,KELT-9,2017,Transit,1.481124,0.03462,21.196,915.3504,2.037415,20.008,44900.0,4050.0,B9.5-A0,10170.0,0.24852,78.91292,666.843022,-654.390943,128.233163


# Part IV: Put forth into javascript

 